In [1]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
from matplotlib import pyplot as plt
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.problems import ElectronicStructureProblem, ElectronicBasis
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import TwoLocal
from circuit_knitting_toolbox.entanglement_forging import EntanglementForgingAnsatz
import numpy as np
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from circuit_knitting_toolbox.entanglement_forging import (
    EntanglementForgingGroundStateSolver,
)
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.algorithms import (
    GroundStateEigensolver,
    NumPyMinimumEigensolverFactory,
)


res = []
# for R in ['0.60','0.70', '0.80', '0.90', '0.91', '0.92', '0.93', '0.94', '0.95', '0.96', '0.97', '0.98', '0.99', '1.00', '1.20', '1.40', '1.60', '1.80', '2.00', '3.00', '4.00']:
for i, R in enumerate(["0.90"]):
    HFs = [-14.09259461609392]
    system_data = np.load(f"{R}_hamiltonian.xyz.npy", allow_pickle=True)[0]
    orb_act = [0, 1, 2, 3, 4]
    num_molecular_orbitals = len(orb_act)
    num_alpha, num_beta = system_data["ne"]
    hcore = system_data["h1"]
    eri = system_data["h2"]
    # mo_coeff = np.eye(hcore.shape[0])

    hamiltonian = ElectronicEnergy.from_raw_integrals(hcore, eri)
    hamiltonian.nuclear_repulsion_energy = system_data["h0"]

    problem = ElectronicStructureProblem(hamiltonian)
    problem.basis = ElectronicBasis.MO
    problem.num_particles = (num_alpha, num_beta)
    converter = QubitConverter(JordanWignerMapper())

    orb_act = [0, 1, 2, 3, 4]
    transformer = ActiveSpaceTransformer(
        num_electrons=6, num_spatial_orbitals=len(orb_act), active_orbitals=orb_act
    )
    problem_reduced = transformer.transform(problem)

    theta = Parameter("θ")

    hop_gate = QuantumCircuit(2, name="hop_gate")
    hop_gate.h(0)
    hop_gate.cx(1, 0)
    hop_gate.cx(0, 1)
    hop_gate.ry(-theta, 0)
    hop_gate.ry(-theta, 1)
    hop_gate.cx(0, 1)
    hop_gate.h(0)

    theta_1, theta_2, theta_3, theta_4 = (
        Parameter("θ1"),
        Parameter("θ2"),
        Parameter("θ3"),
        Parameter("θ4"),
    )

    circuit = QuantumCircuit(5)
    circuit.append(hop_gate.to_gate({theta: theta_1}), [0, 1])
    circuit.append(hop_gate.to_gate({theta: theta_2}), [3, 4])
    circuit.append(hop_gate.to_gate({theta: 0}), [1, 4])
    circuit.append(hop_gate.to_gate({theta: theta_3}), [0, 2])
    circuit.append(hop_gate.to_gate({theta: theta_4}), [3, 4])

    bitstrings = [(1, 1, 1, 0, 0), (0, 1, 1, 0, 1), (1, 1, 0, 1, 0)]
    ansatz = EntanglementForgingAnsatz(
        circuit_u=circuit,
        bitstrings_u=bitstrings,
    )

    optimizer = COBYLA(maxiter=1)
    angles = [-0.83604922, -0.87326138, -0.93964018, 0.55224467]

    for PAR in [1]:
        solver = EntanglementForgingGroundStateSolver(
            ansatz=ansatz,
            optimizer=optimizer,
            hf_energy=HFs[i],
            initial_point=angles,
        )
        res.append(solver.solve(problem_reduced))
        # np.save(
        #    "vqe_forging_%s.npy" % (R),
        #    {
        #        "eigenvalues": [evaluation.eigenvalue for evaluation in res[i].history],
        #        "schmidt": [
        #            abs(evaluation.eigenstate) for evaluation in res[i].history
        #        ],
        #        "parameters": [evaluation.parameters for evaluation in res[i].history],
        #    },
        # )